In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhanes-2015-2016/NHANES.csv


Data explanation : https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIAGENDR

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

path = "../input/nhanes-2015-2016/NHANES.csv"
da = pd.read_csv(path)

In [4]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [5]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      5735 non-null   int64  
 1   ALQ101    5208 non-null   float64
 2   ALQ110    1731 non-null   float64
 3   ALQ130    3379 non-null   float64
 4   SMQ020    5735 non-null   int64  
 5   RIAGENDR  5735 non-null   int64  
 6   RIDAGEYR  5735 non-null   int64  
 7   RIDRETH1  5735 non-null   int64  
 8   DMDCITZN  5734 non-null   float64
 9   DMDEDUC2  5474 non-null   float64
 10  DMDMARTL  5474 non-null   float64
 11  DMDHHSIZ  5735 non-null   int64  
 12  WTINT2YR  5735 non-null   float64
 13  SDMVPSU   5735 non-null   int64  
 14  SDMVSTRA  5735 non-null   int64  
 15  INDFMPIR  5134 non-null   float64
 16  BPXSY1    5401 non-null   float64
 17  BPXDI1    5401 non-null   float64
 18  BPXSY2    5535 non-null   float64
 19  BPXDI2    5535 non-null   float64
 20  BMXWT     5666 non-null   floa

In [6]:
da.isnull().sum()

SEQN           0
ALQ101       527
ALQ110      4004
ALQ130      2356
SMQ020         0
RIAGENDR       0
RIDAGEYR       0
RIDRETH1       0
DMDCITZN       1
DMDEDUC2     261
DMDMARTL     261
DMDHHSIZ       0
WTINT2YR       0
SDMVPSU        0
SDMVSTRA       0
INDFMPIR     601
BPXSY1       334
BPXDI1       334
BPXSY2       200
BPXDI2       200
BMXWT         69
BMXHT         62
BMXBMI        73
BMXLEG       390
BMXARML      308
BMXARMC      308
BMXWAIST     367
HIQ210      1003
dtype: int64

### Problem 1
Restrict the sample to women between 35 and 50, then use the marital status variable DMDMARTL to partition this sample into two groups - women who are currently married, and women who are not currently married. Within each of these groups, calculate the proportion of women who have completed college. Calculate 95% confidence intervals for each of these proportions.

In [7]:
# We need 3 categories : gender woman, age 35-50, marital age
dx = da[['RIAGENDR','RIDAGEYR','DMDMARTL','DMDEDUC2']].dropna()
#Replace some code for easier read
dx.columns = ['Gender', 'Age', 'Marital_status', 'Education']

In [8]:
#Slice column to age 35-50 female
dx['Gender'] = dx['Gender'].replace({1 : 'Male', 2: 'Female'})
dx['Marital_status'] = dx['Marital_status'].replace({1 : 'Married'})
dx['Education'] = dx['Education'].replace({4 : 'College'})
dx = dx[(dx['Gender'] == 'Female')&(dx['Age']>35)&(dx['Age']<50)]
dx.head()

,Gender,Age,Marital_status,Education
4,Female,42,3.0,College
34,Female,37,Married,College
50,Female,39,Married,3.0
55,Female,45,Married,2.0
58,Female,44,5.0,1.0


In [9]:
marital_counts = dx.Marital_status.unique()
other_marital = list(marital_counts[marital_counts != 'Married'])

dx['Marital_status'] = dx['Marital_status'].replace(other_marital, 'Not married')
dx.head()

,Gender,Age,Marital_status,Education
4,Female,42,Not married,College
34,Female,37,Married,College
50,Female,39,Married,3.0
55,Female,45,Married,2.0
58,Female,44,Not married,1.0


In [10]:
college_counts = dx.Education.unique()
non_college = list(college_counts[college_counts != 'College'])

dx['Education'] = dx['Education'].replace(non_college, 'Not college')
dx.head()

,Gender,Age,Marital_status,Education
4,Female,42,Not married,College
34,Female,37,Married,College
50,Female,39,Married,Not college
55,Female,45,Married,Not college
58,Female,44,Not married,Not college


In [11]:
pd.crosstab(dx.Marital_status, dx.Education)

Education,College,Not college
Marital_status,,
Married,117,275
Not married,109,197


In [12]:
dz = dx.groupby(dx.Marital_status).agg({'Education': [lambda x : np.mean(x == 'College'), np.size]})
dz.columns = ['proportion','size']
dz

,proportion,size
Marital_status,,
Married,0.298469,392
Not married,0.356209,306


In [13]:
# Proportion of married (1) women age 35-50 who finished college (4)
proportion_married = dz.iloc[0,0]
proportion_married

0.29846938775510207

In [14]:
# Standard error of married
se_married=proportion_married*(1-proportion_married)
se_married

0.20938541232819657

In [15]:
n_married = dz.iloc[0,1]
n_married

392

In [16]:
# Calculate the boundaries for married female age 35-50 
lbc_married =   proportion_married-1.96*np.sqrt((se_married/n_married))
ubc_married =  proportion_married+1.96*np.sqrt((se_married/n_married))
print('Lower Boundary :',lbc_married)
print('Upper Boundary :',ubc_married)

Lower Boundary : 0.253170634562367
Upper Boundary : 0.34376814094783714


In [17]:
# Proportion of married (1) women age 35-50 who finished college (4)
proportion_not_married = dz.iloc[1,0]
proportion_not_married

0.3562091503267974

In [18]:
# Standard error of not married
se_not_married=proportion_not_married*(1-proportion_not_married)
se_not_married

0.22932419155025843

In [19]:
n_not_married = dz.iloc[1,1]
n_not_married

306

In [20]:
# Calculate the boundaries for married female age 35-50 
lbc_unmarried = proportion_not_married-1.96*np.sqrt((se_not_married/n_not_married))
ubc_unmarried = proportion_not_married+1.96*np.sqrt((se_not_married/n_not_married))
print('Lower Boundary:', lbc_unmarried)
print('Upper Boundary:', ubc_unmarried)

Lower Boundary: 0.3025529031876515
Upper Boundary: 0.4098653974659433


**Q1a.** Identify which of the two confidence intervals is wider, and explain why this is the case.

In [21]:
# Which one is wider?
married = ubc_married - lbc_married
unmarried = ubc_unmarried - lbc_unmarried
print('Married interval:', married)
print('Unmarried interval:', unmarried)

Married interval: 0.09059750638547015
Unmarried interval: 0.10731249427829181


**Answer** : The wider interval of confidence interval is in the unmarried sample. The wider interval happen because the number of sample is fewer and the standard error has a bigger value.

**Conclusion :**


## Problem 2

1. Construct a 95% confidence interval for the proportion of smokers who are female. 
2. Construct a 95% confidence interval for the proportion of smokers who are male. 
3. Construct a 95% confidence interval for the **difference** between those two gender proportions.

In [22]:
# Proportion of female smoker with 95% CI and the difference between proportion of man and woman smoker with 95% CI.

da[['RIAGENDR','SMQ020']].isnull().sum() # no null value, safe
ds = da[['RIAGENDR','SMQ020']]

In [23]:
ds.columns = ['Gender','Smoking']
ds.head()

,Gender,Smoking
0,1,1
1,1,1
2,1,1
3,2,2
4,2,2


In [24]:
ds.Gender.unique()
ds['Gender'] = ds.Gender.replace({1 : 'Male', 2 : 'Female'})
ds.head()

,Gender,Smoking
0,Male,1
1,Male,1
2,Male,1
3,Female,2
4,Female,2


In [25]:
ds.Smoking.unique()
ds['Smoking'] = ds.Smoking.replace({1 : 'Yes', 2 : 'No', 7 : np.nan, 9 : np.nan})
ds.head()

,Gender,Smoking
0,Male,Yes
1,Male,Yes
2,Male,Yes
3,Female,No
4,Female,No


In [26]:
ds.dropna(inplace = True)
ds.head()

,Gender,Smoking
0,Male,Yes
1,Male,Yes
2,Male,Yes
3,Female,No
4,Female,No


In [27]:
pd.crosstab(ds.Smoking, ds.Gender)

Gender,Female,Male
Smoking,,
No,2066,1340
Yes,906,1413


In [28]:
z = ds.groupby(ds.Gender).agg({'Smoking' : [lambda x: np.mean(x == 'Yes'), np.size]})
z.columns = ['proportion', 'size']
z

,proportion,size
Gender,,
Female,0.304845,2972
Male,0.513258,2753


In [29]:
#Female summary
nf = z.iloc[0,1]
pf = z.iloc[0,0] 

In [30]:
# The proportion and lower bound confidence and upper bound confidence for female smoker
import statsmodels.api as sm
bound_f = sm.stats.proportion_confint(nf *pf, nf)
print('proportion of female smoker:', pf)
print('lbc_f:', bound_f[0])
print('ubc_f:', bound_f[1])

proportion of female smoker: 0.30484522207267833
lbc_f: 0.2882949879861214
ubc_f: 0.32139545615923526


In [31]:
# Male summary
nm = z.iloc[1,1]
pm = z.iloc[1,0]

In [32]:
# The proportion and lower bound confidence and upper bound confidence for male smoker
bound_m = sm.stats.proportion_confint(nm *pm, nm)
print('proportion of male smoker:', pm)
print('lbc_f:', bound_m[0])
print('ubc_f:', bound_m[1])

proportion of male smoker: 0.5132582637123139
lbc_f: 0.49458749263718593
ubc_f: 0.5319290347874418


In [33]:
# The difference with interval
difference = pm - pf

# se_diff
se_male = np.sqrt((pm*(1-pm)/nm))
se_female = np.sqrt((pf*(1-pf)/nf))
se_diff = np.sqrt(se_male**2+se_female**2)

#lcb and ucb diff, with 95% CI the z value is 1.96
lcb_diff = difference - (1.96*se_diff)
ucb_diff = difference + (1.96*se_diff)

In [34]:
print('Difference of smoker proportion between male and female:', difference, "±", se_diff)
print('lower bound:', lcb_diff)
print('upper bound:', ucb_diff)

Difference of smoker proportion between male and female: 0.20841304163963553 ± 0.012729881381407434
lower bound: 0.18346247413207697
upper bound: 0.2333636091471941


In [35]:
# Statistical test to determine the difference of two proportion 
ds_females = ds.loc[ds.Gender == 'Female', 'Smoking'].replace({"Yes": 1, "No": 0})
ds_male = ds.loc[ds.Gender == 'Male', 'Smoking'].replace({"Yes": 1, "No": 0})

In [36]:
sm.stats.ttest_ind(ds_females, ds_male)

(-16.42058555898443, 3.032088786691117e-59, 5723.0)

**Q2a** Why might it be relevant to report the separate gender proportions **and** the difference between the gender proportions?

**Q2b.** How does the width of the confidence interval for the difference between the gender proportions compare to the widths of the confidence intervals for the separate gender proportions?

## Problem 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and 

construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [37]:
dh = da['BMXHT'].dropna() #remove the null
dh_inch = dh/2.54 #convert to inch
dh_inch.head()

0    72.637795
1    67.480315
2    66.968504
3    63.346457
4    64.921260
Name: BMXHT, dtype: float64

In [38]:
dh_inch_std = dh_inch.std()
dh_cm_std = dh.std()
n = len(dh)

In [39]:
# Mean in inch
height_inch = dh_inch.mean()
height_inch

65.41056475555557

In [40]:
# SEM height in inch
sem_inch = dh_inch_std/np.sqrt(n)
sem_inch

0.052685208096373254

In [41]:
# CI 95%, z value 1.96
lcb_inch = height_inch - 1.96*sem_inch
ucb_inch = height_inch + 1.96*sem_inch
print(lcb_inch, ucb_inch)

65.30730174768668 65.51382776342446


In [42]:
# 95% confidence for height in inches 
import statsmodels.api as sm
x = sm.stats.DescrStatsW(dh_inch).zconfint_mean()
x

(65.30730364516884, 65.51382586594264)

In [43]:
# Mean in centimeter
height_cm = dh.mean()
height_cm

166.14283447911131

In [44]:
# SEM height in cm
sem_cm = dh_cm_std/np.sqrt(n)
sem_cm

0.13382042856478796

In [45]:
# CI 95%, z value 1.96
lcb_cm = height_cm - 1.96*sem_cm
ucb_cm = height_cm + 1.96*sem_cm
print(lcb_cm, ucb_cm)

165.88054643912432 166.4051225190983


In [46]:
# 95% confidence for height in centimeters
y = sm.stats.DescrStatsW(dh).zconfint_mean()
y

(165.88055125872887, 166.40511769949427)

In [47]:
print('Mean of height in inch:',height_inch, "±", sem_inch)
print('lower bound:',lcb_inch,'upper bound:', ucb_inch)

Mean of height in inch: 65.41056475555557 ± 0.052685208096373254
lower bound: 65.30730174768668 upper bound: 65.51382776342446


In [48]:
print('Mean of height in inch:',height_cm, "±", sem_cm)
print('lower bound:',lcb_cm,'upper bound:', ucb_cm)

Mean of height in inch: 166.14283447911131 ± 0.13382042856478796
lower bound: 165.88054643912432 upper bound: 166.4051225190983


__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

### Problem 4
Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. 
Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [49]:
# Checking the variance between groups
dbmi = da[['RIAGENDR','RIDAGEYR','BMXBMI']].dropna()
dbmi.columns = ['Gender','Age','BMI']
dbmi['Gender'] = dbmi.Gender.replace({1:'Male',2:'Female'})
dbmi['Age'] = pd.cut(dbmi.Age, [18,30,40,50,60,70,80])
dbmi.groupby(['Age','Gender']).agg({'BMI':[np.std, np.size]}).unstack()

BMI                      
               std             size     
Gender      Female      Male Female Male
Age                                     
(18, 30]  7.745893  6.649440    603  538
(30, 40]  8.315608  6.622412    469  452
(40, 50]  8.076195  6.407076    498  398
(50, 60]  7.575848  5.914373    466  451
(60, 70]  7.604514  5.933307    437  426
(70, 80]  6.284968  4.974855    403  390

In [50]:
for j, v in dbmi.groupby('Age'):
    bmi_female = v.loc[v.Gender == 'Female','BMI'].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.Gender == 'Male','BMI'].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    x = sm.stats.CompareMeans(bmi_female, bmi_male).zconfint_diff(usevar='pooled')
    y = sm.stats.CompareMeans(bmi_female, bmi_male).zconfint_diff(usevar='unequal')
    print(j)
    print('pooled:',x)
    print('range:', x[1]-x[0])
    print(j)
    print('pooled:',y)
    print('range:', y[1]-y[0])


(18, 30]
pooled: (-0.11061142651112632, 1.5747294978644097)
range: 1.6853409243755362
(18, 30]
pooled: (-0.10336562645567127, 1.5674836978089546)
range: 1.6708493242646258
(30, 40]
pooled: (-0.2592328949244713, 1.6869542753799731)
range: 1.9461871703044444
(30, 40]
pooled: (-0.25521665952637695, 1.6829380399818787)
range: 1.9381546995082557
(40, 50]
pooled: (0.4632643410694307, 2.4087745683370323)
range: 1.9455102272676017
(40, 50]
pooled: (0.4876803418389215, 2.384358567567541)
range: 1.8966782257286197
(50, 60]
pooled: (0.6306908625345806, 2.393891610367786)
range: 1.7632007478332055
(50, 60]
pooled: (0.634186871588902, 2.3903956013134646)
range: 1.7562087297245625
(60, 70]
pooled: (0.770781604088166, 2.5938986206623347)
range: 1.8231170165741686
(60, 70]
pooled: (0.77360609210102, 2.5910741326494806)
range: 1.8174680405484605
(70, 80]
pooled: (0.3899529786338254, 1.97108920498901)
range: 1.5811362263551847
(70, 80]
pooled: (0.3929267477420184, 1.968115435880817)
range: 1.57518868813

In [51]:
for j, v in dbmi.groupby('Age'):
    bmi_female = v.loc[v.Gender == 'Female','BMI'].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.Gender == 'Male','BMI'].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(j)
    print('pooled:', sm.stats.CompareMeans(bmi_female, bmi_male).zconfint_diff(usevar='pooled'))
    print("unequal:", sm.stats.CompareMeans(bmi_female, bmi_male).zconfint_diff(usevar='unequal'))
    print()      


(18, 30]
pooled: (-0.11061142651112632, 1.5747294978644097)
unequal: (-0.10336562645567127, 1.5674836978089546)

(30, 40]
pooled: (-0.2592328949244713, 1.6869542753799731)
unequal: (-0.25521665952637695, 1.6829380399818787)

(40, 50]
pooled: (0.4632643410694307, 2.4087745683370323)
unequal: (0.4876803418389215, 2.384358567567541)

(50, 60]
pooled: (0.6306908625345806, 2.393891610367786)
unequal: (0.634186871588902, 2.3903956013134646)

(60, 70]
pooled: (0.770781604088166, 2.5938986206623347)
unequal: (0.77360609210102, 2.5910741326494806)

(70, 80]
pooled: (0.3899529786338254, 1.97108920498901)
unequal: (0.3929267477420184, 1.968115435880817)



__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see. How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

In pooled or non-pooled method there are difference in confidence interval. The age range with biggest range intervals is age 30-40 

### Problem 5
Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

**Q5a**. Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

**Q5b**. Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

### Problem 6
Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

**Q6a** Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.

**Q6b** Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?